In [1]:
import numpy as np
import cv2
import sys
from random import randint
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
video_source = 'Pedestrians_2.mp4'

In [3]:
text_color = (randint(0,255),randint(0,255),randint(0,255))
border_color = (randint(0,255),randint(0,255),randint(0,255))
bgs_types = ['MOG','MOG2','GNG','KNN','CNT']
bg = bgs_types[0]

In [4]:
def get_kernel(kernel_type):
    
    if kernel_type == 'dilation':
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    if kernel_type =='opening':
        kernel = np.ones((3,3),np.uint8)
    if kernel_type =='closing':
        kernel = np.ones((3,3),np.uint8)
    return kernel

In [5]:
def get_filters(img,img_filter):
    if img_filter == 'closing':
        return cv2.morphologyEx(img,cv2.MORPH_CLOSE,get_kernel('closing'),iterations=2)
    if img_filter == 'opening':
        return cv2.morphologyEx(img,cv2.MORPH_OPEN,get_kernel('opening'),iterations=2)
    if img_filter == 'dilation':
        return cv2.dilate(img,get_kernel('dilation'),iterations=2)
    if img_filter == 'combine':
        closing = cv2.morphologyEx(img,cv2.MORPH_CLOSE,get_kernel('closing'),iterations=2)
        opening = cv2.morphologyEx(closing,cv2.MORPH_OPEN,get_kernel('opening'),iterations=2)
        dilation = cv2.dilate(img,get_kernel('dilation'),iterations=2)
        return dilation

In [6]:
def get_bgsubtractor(bgs_type):
    if bgs_type =='GMG':
        return cv2.bgsegm.createBackgroundSubtractorGMG()
    if bgs_type =='MOG':
        return cv2.bgsegm.createBackgroundSubtractorMOG()        
    if bgs_type =='MOG2':
        return cv2.createBackgroundSubtractorMOG2(varThreshold = 100,detectShadows = False)
    if bgs_type =='KNN':
        return cv2.createBackgroundSubtractorKNN()        
    if bgs_type =='CNT':
        return cv2.bgsegm.createBackgroundSubtractorCNT()
    print('Invalid Detetctor')
    sys.exit(0)
        


In [60]:
cap = cv2.VideoCapture(video_source)
bg_subtractor = get_bgsubtractor(bg)
minArea = 400
maxArea = 3000

In [61]:
def main():
    while cap.isOpened():
        ok,frame = cap.read()
        
        if not ok:
            print('video is ended')
            break
        
        frame = cv2.resize(frame,(0,0), fx=0.5, fy=0.5)
        bg_mask = bg_subtractor.apply(frame)
        bg_mask = get_filters(bg_mask,'combine')
        bg_mask = cv2.medianBlur(bg_mask,5)
        
        contours,hierarchy = cv2.findContours(bg_mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area >= minArea:
                x,y,w,h = cv2.boundingRect(cnt)
                cv2.drawContours(frame,cnt,1,(255,0,0),10)
                
                if area >= maxArea:
                    cv2.rectangle(frame,(x,y),(x+120,y-13),(49,49,49),-1)
                    cv2.putText(frame,'Warning',(x,y-2),cv2.FONT_HERSHEY_COMPLEX,0.4,(255,255,255),1,cv2.LINE_AA)
                    cv2.drawContours(frame,[cnt],-1,(0,0,255),2)
        
        cv2.imshow('original',frame)
        cv2.imshow('bg_mask', bg_mask)
        k = cv2.waitKey(1) and 0xff
        if k==(27):
            print('done')
            break

In [62]:
main()

video is ended
